In [204]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import re

In [2]:
from itertools import product, chain
from random import choices, shuffle

In [3]:
ALL_SUITS = list(range(5))
ALL_CARDS = [(x,y) for (x,y) in product(ALL_SUITS, range(9)) if x>0 or y<4]
ALL_TASKS = list(ALL_CARDS[4:])

In [4]:
CARD_INDICER = {x:i for i, x in enumerate(ALL_CARDS)}
TASK_INDICER = {x:i for i, x in enumerate(ALL_TASKS)}

In [5]:
N = 4
M = 2

In [10]:
class AssignerNet(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(len(TASK_INDICER)*(n+1)+n+len(CARD_INDICER), 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, len(TASK_INDICER)),
            nn.Softmax(dim=1)
        )
        self.float()

    def forward(self, x):
        logits = self.layers(x)
        return logits

class Assigner:
    
    def __init__(self, n, m):
        self.n = n
        self.m = m
        self.nn = AssignerNet(n)
        self.loss_fn = nn.MSELoss().float()
    
    def assign(self, hands, tasks):
        lead = next(i for i in range(self.n) if (0,3) in hands[i])
        chooser = lead
        task_mask = np.zeros(len(TASK_INDICER), dtype=bool)
        task_assignment_masks = np.zeros((self.n, len(TASK_INDICER)), dtype=bool)
        lead_mask = np.zeros(self.n, dtype=bool)
        hand_masks = np.zeros((self.n, len(CARD_INDICER)), dtype=bool)
        
        self.nn.eval()
        
        for t in tasks:
            task_mask[TASK_INDICER[t]] = 1
        
        for i in range(self.n):
            for card in hands[i]:
                hand_masks[i][CARD_INDICER[card]] = 1

        for _ in range(self.m):
            lead_mask[(lead-chooser+self.n)%self.n] = 1
            inp = np.concatenate((
                task_mask,
                task_assignment_masks[chooser:].flatten(),
                task_assignment_masks[:chooser].flatten(),
                lead_mask,
                hand_masks[chooser]
            )).reshape((1, -1))
            with torch.no_grad():
                out = self.nn(torch.tensor(inp, dtype=torch.float)).detach().numpy().flatten()
            out[~task_mask] = 0.
            out[task_assignment_masks.sum(axis=0).astype(bool)] = 0.
            # print(out)
            # print(task_mask.astype(int))
            # print(task_assignment_masks.astype(int))
            # print(task_assignment_masks.sum(axis=0).astype(int))
            # print()
            k = np.argmax(out)
            task_assignment_masks[chooser][k] = 1
            lead_mask[(lead-chooser+self.n)%self.n] = 0
            chooser = (chooser+1)%self.n
        
        return task_assignment_masks

    def learn_from(self, hands, tasks, task_assignment_masks, won_cards_mask):
        lead = next(i for i in range(self.n) if (0,3) in hands[i])
        chooser = lead
        task_mask = np.zeros(len(TASK_INDICER), dtype=bool)
        task_assignment_masks_partial = np.zeros((self.n, len(TASK_INDICER)), dtype=bool)
        lead_mask = np.zeros(self.n, dtype=bool)
        hand_masks = np.zeros((self.n, len(CARD_INDICER)), dtype=bool)
        
        for t in tasks:
            task_mask[TASK_INDICER[t]] = 1
        
        for i in range(self.n):
            for card in hands[i]:
                hand_masks[i][CARD_INDICER[card]] = 1

        for _ in range(self.m):
            lead_mask[(lead-chooser+self.n)%self.n] = 1
            inp = np.concatenate((
                task_mask,
                task_assignment_masks_partial[chooser:].flatten(),
                task_assignment_masks_partial[:chooser].flatten(),
                lead_mask,
                hand_masks[chooser]
            )).reshape((1, -1))
            out = self.nn(torch.tensor(inp, dtype=torch.float)).flatten()
            target = (won_cards_mask[chooser][4:] & task_mask)
            target = torch.tensor(target/(1e-6 + target.sum()), dtype=torch.float)
            loss = self.loss_fn(out, target)
            loss.backward()
            # print(out)
            # print(task_mask.astype(int))
            # print(task_assignment_masks.astype(int))
            # print(task_assignment_masks.sum(axis=0).astype(int))
            # print()

            task_assignment_masks_partial[chooser] = task_assignment_masks[chooser]
            lead_mask[(lead-chooser+self.n)%self.n] = 0
            chooser = (chooser+1)%self.n
        
        return task_assignment_masks_partial

In [11]:
# class PlayerNet(nn.Module):
#     def __init__(self, n):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(len(TASK_INDICER)*(n+1)+n+len(CARD_INDICER), 512),
#             nn.ReLU(),
#             nn.Linear(512, 512),
#             nn.ReLU(),
#             nn.Linear(512, len(CARD_INDICER)),
#             nn.Sigmoid(),
#         )

#     def forward(self, x):
#         raise NotImplementedError
#         return self.layers(x)

# class Player:
    
#     def __init__(self, n, m):
#         self.n = n
#         self.m = m
#         self.nn = PlayerNet(n)
    
#     def assign(self, hands, tasks):
#         raise NotImplementedError
#         lead = next(i for i in range(self.n) if (0,3) in hands[i])
#         chooser = lead
#         task_mask = np.zeros(len(TASK_INDICER), dtype=bool)
#         task_assignment_masks = np.zeros((self.n, len(TASK_INDICER)), dtype=bool)
#         lead_mask = np.zeros(self.n, dtype=bool)
#         hand_masks = np.zeros((self.n, len(CARD_INDICER)), dtype=bool)

#         for t in tasks:
#             task_mask[TASK_INDICER[t]] = 1

#         for i in range(self.n):
#             for card in hands[i]:
#                 hand_masks[i][CARD_INDICER[card]] = 1

#         for _ in range(self.m):
#             lead_mask[(lead-chooser+self.n)%self.n] = 1
#             inp = np.concatenate((
#                 task_mask,
#                 task_assignment_masks[chooser:].flatten(),
#                 task_assignment_masks[:chooser].flatten(),
#                 lead_mask,
#                 hand_masks[chooser]
#             ))
#             out = self.nn(torch.tensor(inp, dtype=torch.float)).detach().numpy().flatten()
#             out[~task_mask] = 0.
#             out[task_assignment_masks.sum(axis=0).astype(bool)] = 0.
#             # print(out)
#             # print(task_mask.astype(int))
#             # print(task_assignment_masks.astype(int))
#             # print(task_assignment_masks.sum(axis=0).astype(int))
#             # print()
#             k = np.argmax(out)
#             task_assignment_masks[chooser][k] = 1
#             lead_mask[(lead-chooser+self.n)%self.n] = 0
#             chooser = (chooser+1)%self.n

#         return task_assignment_masks

In [12]:
class Game:
    def __init__(self, n, m, assigner=None):
        self.n = n
        self.m = m
        self.tasks = choices(ALL_TASKS, k=m)
        self.initialize_hands()
        self.lead = next(i for i in range(self.n) if (0,3) in self.hands[i])
        self.assigner = assigner if assigner else Assigner(self.n, self.m)
    
    def assign_tasks(self):
        self.task_assignment_masks = self.assigner.assign(self.hands, self.tasks)
        # print(self.task_assignment_masks.astype(int))
        # print()
    
    def initialize_hands(self):
        deck = list(ALL_CARDS)
        shuffle(deck)
        self.hands = []
        self.initial_hands = []
        for i in range(self.n):
            self.hands.append(deck[i*(40//self.n):(i+1)*(40//self.n)])
            self.initial_hands.append(deck[i*(40//self.n):(i+1)*(40//self.n)])
    
    def play_randomly(self):
        self.won_cards = [[] for _ in range(self.n)]
        self.task_completion_masks = np.zeros((self.n, len(TASK_INDICER)), dtype=bool)
        lead = self.lead
        for _ in range(40//self.n):
            trick = choices(self.hands[lead])
            # print(trick, self.hands[lead])
            self.hands[lead].remove(trick[0])
            w = 0
            for j in range(1, self.n):
                options = [c for c in self.hands[(lead+j)%self.n] if c[0]==trick[0][0]]
                if not options:
                    options = self.hands[(lead+j)%self.n]
                trick.extend(choices(options))
                self.hands[(lead+j)%self.n].remove(trick[-1])
                if (trick[j][0] == trick[w][0] and trick[j][1] > trick[w][1]) or (trick[j][0] == 0 < trick[w][0]):
                    w = j
            w = (w + lead) % self.n
            self.won_cards[w].extend(trick)
            for c in trick:
                if c[0] > 0:
                    t = TASK_INDICER[c]
                    self.task_completion_masks[w][t] = self.task_assignment_masks[:, t].sum()
            lead = w
        
        self.score = (self.task_assignment_masks & self.task_completion_masks).sum() / self.m
        # print(self.task_assignment_masks.astype(int))
        # print(self.task_completion_masks.astype(int))
        # print()

    def teach_assigner(self):
        won_cards_mask = np.zeros((self.n, len(CARD_INDICER)), dtype=bool)
        for i in range(self.n):
            for card in self.won_cards[i]:
                won_cards_mask[i, CARD_INDICER[card]] = 1
        
        self.assigner.learn_from(self.initial_hands, self.tasks, self.task_assignment_masks, won_cards_mask)

#         assert sorted(list(chain(*self.hands)) + list(chain(*self.won_cards))) == ALL_CARDS, (sorted(list(chain(*self.hands))), ALL_CARDS, self.hands)


In [13]:
assigner = None

In [14]:
games = []
for _ in range(1000):
    game = Game(N, M, assigner=assigner)
    assigner = game.assigner
    game.assign_tasks()
    game.play_randomly()
    game.teach_assigner()
    games.append(game)

scores = [game.score for game in games]

In [15]:
print(np.mean(scores))

0.284


In [29]:
username_map = {'Funwithmonsters': 'JohnnyB1909', 'christopherenr': 'Entropyzer'}

In [225]:
from dataclasses import dataclass
from multidict import MultiDict
from itertools import chain
import re

@dataclass
class Player:
    player_name: str
    player_id: int


class Hand(list):
    def __init__(self, hand_text):
        suit_hands = [y.split(' ') for y in hand_text.strip(' \r\n').replace(' \r', '').split('\n')]
        super().__init__(sum(([(5-int(sh[-1]), int(c)-1) for c in sh[:-1]] for sh in suit_hands), start=[]))
    def __repr__(self):
        return "[" + " ".join([f"{card[0]}_{card[1]}" for card in self]) + "]"


class Task(tuple):
    def __new__(cls, task_text):
        return super().__new__(cls, (5-int(task_text[1]), int(task_text[0])-1))

class MissionLogs():
    trick_search_regex = "([^\r\n]+) wins the trick:([\s\d]+)"

    def __init__(self, mission_id, mission_logs_text, mission_status, starting_player_hands, players):
        self.mission_id = int(mission_id)
        self.mission_logs_text = mission_logs_text
        self.mission_success = (mission_status == 'completed')
        self.starting_player_hands = starting_player_hands

        self.task_assignment = MultiDict([(player_name, Task(task_text)) for player_name, task_text in re.findall('([^\r\n]+) takes task (\d+)', mission_logs_text)])
        self.commander = re.match("([^\r\n]+) is your new commander", mission_logs_text)[1]

        if not (known_players := {p.player_name for p in players}).issuperset(found_players := {username_map.get(k, k) for k in self.task_assignment}):

            if len(taskless_players := known_players - found_players) == len(unknown_players := found_players - known_players) == 1:
                username_map[next(iter(unknown_players))] = next(iter(taskless_players))
            else:
                raise Exception(f"{unknown_players=} {taskless_players=}")


        self.won_tricks = MultiDict([(username_map.get(player_name, player_name), Hand(trick_text)) for (player_name, trick_text) in re.findall(MissionLogs.trick_search_regex, mission_logs_text)])


    def is_successful(self):

        for p in self.task_assignment:
            tasks_p = set(self.task_assignment.getall(p, []))
            won_cards_p = set(chain(*self.won_tricks.getall(p, [])))
            if not set(tasks_p).issubset(won_cards_p):
                self.failing_player = p
                return False

        return True


class GameLogs():
    deal_search_regex = "?".join(["(?:-- Your cards are:([\d\s]+))"]*5).replace(")?(", ")(", 1) + "?"
    mission_search_regex = "Start new mission (\d+)\r\n([^~]+?)\r\n(?:Mission \\1 (completed|failed)|All players agreed to fail mission \\1)"

    def __init__(self, game_id, combined_game_logs_text):
        self.game_id = game_id
        player_descriptions, combined_mission_logs = combined_game_logs_text.split('--\r\n', maxsplit=1)
        self.players = [Player(player_name, player_id) for (player_name, player_id) in re.findall("(.+)@(.+)\r\n", player_descriptions)]
        if len(self.players) < 3:
            return

        num_deals = len(re.findall(type(self).deal_search_regex, combined_mission_logs))
        num_missions = len(re.findall(type(self).mission_search_regex, combined_mission_logs))
        
        if num_deals < num_missions:
            print(f"{num_deals=} {len(self.players)=} {num_missions=}")
            print(*re.findall(type(self).deal_search_regex, combined_mission_logs))
            print(f"{self.players=}")
            print([(mission_id, mission_status) for (mission_id, mission_logs_text, mission_status) in re.findall(type(self).mission_search_regex, combined_mission_logs)])
        assert num_deals >= num_missions, f"{num_deals=} {len(self.players)=} {num_missions=}"

        starting_deals_i = iter(re.findall(type(self).deal_search_regex, combined_mission_logs))
        self.combined_mission_logs = []
        if len(re.findall(type(self).mission_search_regex, combined_mission_logs)) == 0:
            print(f"{combined_mission_logs=}\n{combined_game_logs_text=}")
            raise Exception("NO MISSIONS?")
        for (mission_id, mission_logs_text, mission_status) in re.findall(type(self).mission_search_regex, combined_mission_logs):
            mission_id = int(mission_id)
            try:
                starting_deal = next(starting_deals_i)
                while len([hand_text for hand_text in starting_deal if hand_text!=""]) == 1:
                    starting_deal = next(starting_deals_i)
                if mission_id not in [12, 16,26,34]:
                    try:
                        starting_player_hands = [Hand(hand_text) for hand_text in starting_deal if hand_text != ""]
                        if len(starting_player_hands) == len(self.players) - 1:
                            self.players = self.players[:-1]
                        assert len(starting_player_hands) == len(self.players), f"{starting_player_hands=} {len(self.players)=}"
                        mission_logs = MissionLogs(mission_id, mission_logs_text, mission_status, starting_player_hands, self.players)
                        self.combined_mission_logs.append(mission_logs)

                    except StopIteration as exception:
                        hands = [hand for hand in next(starting_deals_i) if hand != ""]
                        print(game_id, mission_id, f"{num_deals=} {hands=} {combined_game_logs_text=}")
            except Exception as exception:
                    print(game_id, mission_id, f"{exception=}")
                    # print(combined_game_logs_text)
                    # print("]\n")




In [226]:
print(len(re.findall("(?:-- Your cards are:([\d\s]+))(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?", game_logs_text)))
print(len(re.findall(GameLogs.deal_search_regex, game_logs_text)))
print(GameLogs.deal_search_regex)
print("(?:-- Your cards are:([\d\s]+))(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?")
# print(game_logs_text)

0
0
(?:-- Your cards are:([\d\s]+))(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?
(?:-- Your cards are:([\d\s]+))(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?(?:-- Your cards are:([\d\s]+))?


In [227]:

if __name__ == '__main__':
    input_path = "/mnt/c/Users/meet1/Code/BGA_Spider/others_1248.txt"
    game_logs_list = []
    with open(input_path, "rb") as replay_file:
        full_text = replay_file.read().decode("ISO-8859-1")

    game_id_matches = list(re.finditer("https://boardgamearena\.com/table\?table=(\d+)", full_text))

    print(f"Found logs for {len(game_id_matches)} games")

    for i, m in enumerate(game_id_matches):

        # try:
            game_id = int(m[1])
            gl_end = game_id_matches[i+1].span()[0] if i+1 < len(game_id_matches) else None
            game_logs_text = full_text[m.span()[1]:gl_end].strip('\r\n')
            game_logs = GameLogs(game_id, game_logs_text)
            if len(game_logs.players) >= 3:
                game_logs_list.append(game_logs)
        # except Exception as e:
        #     print(f"{game_id=}")
        #     print("game_logs_text=[")
        #     print(game_logs_text)
        #     print("]")
        #     raise e


Found logs for 20111 games
combined_mission_logs='\r\n\r\n==='
combined_game_logs_text='shira133@85120855\r\nszvemese@89344963\r\nCR31@94510984\r\nQuort@96119715\r\n--\r\n\r\n\r\n==='


Exception: NO MISSIONS?

In [214]:
len(game_logs_list), sum(len(game_logs.combined_mission_logs) for game_logs in game_logs_list)

(19926, 11838)

In [218]:
# [len(game_logs.combined_mission_logs) for game_logs in game_logs_list]
len([i for i, game_logs in enumerate(game_logs_list) if len(game_logs.combined_mission_logs)==0])

18786

In [456]:
self = next(chain(*((mission_logs for mission_logs in asdc.combined_mission_logs if mission_logs.is_successful() != mission_logs.mission_success and mission_logs.mission_success) for asdc in game_logs_list)))

for p in self.task_assignment:
    tasks_p = set(self.task_assignment.getall(p, []))
    won_cards_p = set(chain(*self.won_tricks.getall(p, [])))
    if not set(tasks_p).issubset(won_cards_p):
        self.failing_player = p
        break

In [461]:
# set(self.task_assignment.getall('Gerrete', []))
set(chain(*self.won_tricks.getall('Pinkmafia', [])))
# self.mission_success

{(1, 0),
 (1, 2),
 (1, 7),
 (2, 5),
 (2, 8),
 (3, 1),
 (3, 2),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7)}

In [462]:
set(self.task_assignment.getall('Pinkmafia', []))

{(3, 4), (4, 4)}

In [451]:
self.mission_id

34

In [443]:
set(chain(*self.won_tricks.getall('Gerrete', [])))

{(1, 0), (1, 2), (1, 7), (1, 8), (2, 6), (4, 3), (4, 6), (4, 8)}

In [318]:
print(mission_logs)

Gerrete is your new commander
Move 2 :
12:05:15 PM
Gerrete takes task 84
Move 3 :
12:05:21 PM
kathkiwi56 takes task 21
Move 4 :
12:05:41 PM
Arang777 takes task 51
Move 5 :
12:05:57 PM
Pitti42 takes task 74
Selecting the last task automatically
Gerrete takes task 91
Move 11 :
12:06:30 PM
Cards will be turned clockwise
Move 12 :
12:06:33 PM
You chose to give the 74
Move 13 :
12:06:36 PM
You chose to give the 71
Move 14 :
12:06:36 PM
You chose to give the 32
Move 15 :
12:06:54 PM
You chose to give the 61
You receive 74 from Arang777
You give 61 to Gerrete
You give 74 to Pitti42
You receive 32 from kathkiwi56
You receive 61 from Pitti42
You give 71 to kathkiwi56
You receive 71 from Gerrete
You give 32 to Arang777
Move 16 :
12:07:08 PM
Gerrete plays 84
Move 17 :
12:07:10 PM
kathkiwi56 plays 14
Move 18 :
12:07:19 PM
Arang777 plays 73
Move 19 :
12:07:21 PM
Pitti42 plays 34
Gerrete wins the trick: 
7 3 
1 3 8 4
Gerrete fulfilled task 84
Move 20 :
12:07:25 PM
Gerrete plays 91
Move 21 :
12:07:27

<MultiDict('Gerrete': [(2, 6), (1, 0), (1, 2), (1, 7)], 'Gerrete': [(4, 3), (4, 6), (4, 8), (1, 8)], 'Arang777': [(4, 4), (4, 7), (3, 5), (1, 3)], 'kathkiwi56': [(4, 1), (4, 5), (3, 0), (0, 1)], 'Pitti42': [(3, 6), (2, 5), (1, 5), (1, 6)])>

In [141]:
# print(combined_mission_logs)

len(mission_statuses)

17

[('10',
  'Gerrete is your new commander\r\nMove 2 :\r\n4:23:34 PM\r\nGerrete takes task 52\r\nMove 3 :\r\n4:23:45 PM\r\nvizslagal takes task 13\r\nMove 4 :\r\n4:24:07 PM\r\njonjamal takes task 74\r\nSelecting the last task automatically\r\nStraegr takes task 72\r\nMove 7 :\r\n4:24:15 PM\r\nNo cards will be passed\r\nMove 8 :\r\n4:24:19 PM\r\nGerrete plays 72\r\nMove 9 :\r\n4:24:24 PM\r\nvizslagal plays 22\r\nMove 10 :\r\n4:24:28 PM\r\njonjamal plays 82\r\nMove 11 :\r\n4:24:31 PM\r\nStraegr plays 92\r\nStraegr wins the trick: \r\n2 7 8 9 2\r\nStraegr fulfilled task 72\r\nMove 12 :\r\n4:24:37 PM\r\nStraegr plays 13\r\nMove 13 :\r\n4:24:44 PM\r\nGerrete plays 53\r\nMove 14 :\r\n4:25:10 PM\r\nvizslagal plays 93\r\nMove 15 :\r\n4:25:13 PM\r\njonjamal plays 44\r\nvizslagal wins the trick: \r\n1 5 9 3 \r\n4 4\r\nvizslagal fulfilled task 13\r\nMove 16 :\r\n4:25:19 PM\r\nvizslagal plays 74\r\nMove 17 :\r\n4:25:22 PM\r\njonjamal plays 84\r\nMove 18 :\r\n4:25:27 PM\r\nStraegr plays 31\r\nMove 19

In [159]:
combined_mission_logs

'Move 1 :\r\n8/14/2024 4:23:22 PM\r\n-- Your cards are: \r\n5 8 9 1 \r\n1 2 2 \r\n9 3 \r\n5 7 4 \r\n1 2 5\r\n-- Your cards are: \r\n7 1 \r\n4 8 2 \r\n1 2 3 4 6 8 9 4\r\n-- Your cards are: \r\n1 3 1 \r\n3 5 6 9 2 \r\n1 2 3 6 3\r\n-- Your cards are: \r\n2 4 6 1 \r\n7 2 \r\n4 5 7 8 3 \r\n3 4 5\r\nStart new mission 10\r\nGerrete is your new commander\r\nMove 2 :\r\n4:23:34 PM\r\nGerrete takes task 52\r\nMove 3 :\r\n4:23:45 PM\r\nvizslagal takes task 13\r\nMove 4 :\r\n4:24:07 PM\r\njonjamal takes task 74\r\nSelecting the last task automatically\r\nStraegr takes task 72\r\nMove 7 :\r\n4:24:15 PM\r\nNo cards will be passed\r\nMove 8 :\r\n4:24:19 PM\r\nGerrete plays 72\r\nMove 9 :\r\n4:24:24 PM\r\nvizslagal plays 22\r\nMove 10 :\r\n4:24:28 PM\r\njonjamal plays 82\r\nMove 11 :\r\n4:24:31 PM\r\nStraegr plays 92\r\nStraegr wins the trick: \r\n2 7 8 9 2\r\nStraegr fulfilled task 72\r\nMove 12 :\r\n4:24:37 PM\r\nStraegr plays 13\r\nMove 13 :\r\n4:24:44 PM\r\nGerrete plays 53\r\nMove 14 :\r\n4:25:10

In [121]:
print(game_logs_list['550722071'])

kathkiwi56@88986947
Gerrete@92803404
Arang777@93899723
Pitti42@94668522
--
Move 1 :
8/16/2024 12:05:03 PM
-- Your cards are: 
2 4 8 1 
2 9 2 
4 6 7 3 
7 4 
1 5
-- Your cards are: 
6 1 
1 6 8 2 
5 8 3 
2 3 9 4 
3 5
-- Your cards are: 
1 3 5 7 9 1 
4 5 7 2 
8 4 
4 5
-- Your cards are: 
3 2 
1 2 3 9 3 
1 4 5 6 4 
2 5
Start new mission 21
Gerrete is your new commander
Move 2 :
12:05:15 PM
Gerrete takes task 84
Move 3 :
12:05:21 PM
kathkiwi56 takes task 21
Move 4 :
12:05:41 PM
Arang777 takes task 51
Move 5 :
12:05:57 PM
Pitti42 takes task 74
Selecting the last task automatically
Gerrete takes task 91
Move 11 :
12:06:30 PM
Cards will be turned clockwise
Move 12 :
12:06:33 PM
You chose to give the 74
Move 13 :
12:06:36 PM
You chose to give the 71
Move 14 :
12:06:36 PM
You chose to give the 32
Move 15 :
12:06:54 PM
You chose to give the 61
You receive 74 from Arang777
You give 61 to Gerrete
You give 74 to Pitti42
You receive 32 from kathkiwi56
You receive 61 from Pitti42
You give 71 to kathkiw